In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("unique_trial_data.csv")
df.head() # Loading the Raw Data

,nct_id,phase,condition,intervention_type,study_design,sponsor_type,enrollment,enrollment_type,status,gender,location,start_date,completion_date,masking
0,NCT00000102,PHASE1/PHASE2,Congenital Adrenal Hyperplasia,DRUG,NaN,NIH,NaN,NaN,COMPLETED,ALL,South Carolina,NaN,NaN,DOUBLE
1,NCT00000104,NaN,Lead Poisoning,PROCEDURE,NaN,FED,NaN,NaN,COMPLETED,FEMALE,Minnesota,NaN,NaN,NaN
2,NCT00000105,NaN,Cancer,BIOLOGICAL,NaN,OTHER,112.0,ACTUAL,TERMINATED,ALL,Minnesota,2002-07-31,2012-03-31,NaN
3,NCT00000106,NaN,Rheumatic Diseases,DEVICE,RANDOMIZED,NIH,NaN,NaN,UNKNOWN,ALL,Wisconsin,NaN,NaN,NaN
4,NCT00000107,NaN,"Heart Defects, Congenital",NaN,NaN,NIH,NaN,NaN,COMPLETED,ALL,Vermont,NaN,NaN,NaN


In [ ]:
df = df.drop('nct_id', axis=1) 

In [ ]:
df['study_design'].fillna('UNKNOWN', inplace=True) # Replace NaN values with 'UNKNOWN'

/tmp/ipykernel_55345/898100831.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['study_design'].fillna('UNKNOWN', inplace=True)


In [79]:
df['study_design'].value_counts()

study_design
RANDOMIZED        154593
UNKNOWN           118006
NON_RANDOMIZED     27401
Name: count, dtype: int64

In [80]:
df.dropna(subset=['phase'], inplace=True)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142095 entries, 0 to 299998
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   phase              142095 non-null  object 
 1   condition          142095 non-null  object 
 2   intervention_type  142095 non-null  object 
 3   study_design       142095 non-null  object 
 4   sponsor_type       142095 non-null  object 
 5   enrollment         139148 non-null  float64
 6   enrollment_type    132310 non-null  object 
 7   status             142095 non-null  object 
 8   gender             142009 non-null  object 
 9   location           91340 non-null   object 
 10  start_date         140052 non-null  object 
 11  completion_date    133327 non-null  object 
 12  masking            137863 non-null  object 
dtypes: float64(1), object(12)
memory usage: 15.2+ MB


In [82]:
df = df[df['status'].isin(['COMPLETED', 'TERMINATED', 'WITHDRAWN'])]

In [83]:
df['status'].value_counts()

status
COMPLETED     100970
TERMINATED     15325
WITHDRAWN       5221
Name: count, dtype: int64

In [84]:
df.dropna(subset=['masking'], inplace=True)

In [85]:
df.dropna(subset=['gender'], inplace=True)

In [86]:
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['completion_date'] = pd.to_datetime(df['completion_date'], errors='coerce')
df['duration'] = (df['completion_date'] - df['start_date']).dt.days
df.drop(columns=['start_date', 'completion_date'], inplace=True)

In [87]:
mean_enrollment = df['enrollment'].mean()
mean_duration = df['duration'].mean()

# Find rows where enrollment is NaN
enrollment_na_mask = df['enrollment'].isna()

# Fill enrollment NaNs with mean
df.loc[enrollment_na_mask, 'enrollment'] = mean_enrollment

# Overwrite enrollment_type to 'ESTIMATED' where enrollment was replaced
df.loc[enrollment_na_mask, 'enrollment_type'] = 'ESTIMATED'

# Fill duration NaNs with mean
df['duration'] = df['duration'].fillna(mean_duration)

In [88]:
df['location'] = df['location'].fillna("other")

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117741 entries, 0 to 299998
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   phase              117741 non-null  object 
 1   condition          117741 non-null  object 
 2   intervention_type  117741 non-null  object 
 3   study_design       117741 non-null  object 
 4   sponsor_type       117741 non-null  object 
 5   enrollment         117741 non-null  float64
 6   enrollment_type    112711 non-null  object 
 7   status             117741 non-null  object 
 8   gender             117741 non-null  object 
 9   location           117741 non-null  object 
 10  masking            117741 non-null  object 
 11  duration           117741 non-null  float64
dtypes: float64(2), object(10)
memory usage: 11.7+ MB


In [90]:
df.loc[df['status']=='WITHDRAWN', 'status'] = 'TERMINATED'

In [91]:
df.to_csv("For_EDA.csv", index=False)

In [43]:
df['status'].value_counts()

status
COMPLETED     97525
TERMINATED    20216
Name: count, dtype: int64

In [44]:
from sklearn.preprocessing import LabelEncoder

# Create a copy of df to store encoded data
df_encoded = df.copy()

# Columns to encode (excluding 'status', 'enrollment', and 'duration')
cols_to_encode = [col for col in df.columns if col not in ['status', 'enrollment', 'duration']]

# Store label encoders for each column
label_encoders = {}

for col in cols_to_encode:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le

In [65]:
df_encoded['status'].replace({'COMPLETED': 0, 'TERMINATED': 1}, inplace=True)

/tmp/ipykernel_55345/3628498710.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_encoded['status'].replace({'COMPLETED': 0, 'TERMINATED': 1}, inplace=True)
/tmp/ipykernel_55345/3628498710.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_encoded['status'].replace({'COMPLETED': 0, 'TERMINA

In [66]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 117741 entries, 0 to 299998
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   phase              117741 non-null  int64  
 1   condition          117741 non-null  int64  
 2   intervention_type  117741 non-null  int64  
 3   study_design       117741 non-null  int64  
 4   sponsor_type       117741 non-null  int64  
 5   enrollment         117741 non-null  float64
 6   enrollment_type    117741 non-null  int64  
 7   status             117741 non-null  int64  
 8   gender             117741 non-null  int64  
 9   location           117741 non-null  int64  
 10  masking            117741 non-null  int64  
 11  duration           117741 non-null  float64
dtypes: float64(2), int64(10)
memory usage: 11.7 MB


In [67]:
df_encoded.to_csv("Encoded_without_sampling.csv", index=False)

In [68]:
df_encoded['status'].value_counts()

status
0    97525
1    20216
Name: count, dtype: int64

In [69]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df_encoded[df_encoded['status'] == 0]
df_minority = df_encoded[df_encoded['status'] == 1]

# Undersample majority class to 50000 samples
df_majority_undersampled = resample(
    df_majority,
    replace=False,
    n_samples=50000,
    random_state=42
)

# Oversample minority class to 50000 samples
df_minority_oversampled = resample(
    df_minority,
    replace=True,
    n_samples=50000,
    random_state=42
)

# Combine undersampled majority and oversampled minority
df_balanced_sampled = pd.concat([df_majority_undersampled, df_minority_oversampled], ignore_index=True)

# Shuffle the resulting dataframe
df_balanced_sampled = df_balanced_sampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [73]:
df_balanced_sampled['status'].value_counts()

status
1    50000
0    50000
Name: count, dtype: int64

In [74]:
df_balanced_sampled.to_csv("Encoded_sampled.csv", index=False)

In [46]:
completed_sample = df[df['status'] == 'COMPLETED'].sample(n=20000, random_state=42)

terminated_all = df[df['status'] == 'TERMINATED']
withdrawn_all = df[df['status'] == 'WITHDRAWN']

df_balanced = pd.concat([completed_sample, terminated_all, withdrawn_all], ignore_index=True)

In [47]:
df_balanced['final_status'] = np.where(df_balanced['status'] == 'COMPLETED', 1, 0)

In [48]:
df_balanced = df_balanced.drop('status', axis=1)

In [49]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40216 entries, 0 to 40215
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   phase              40216 non-null  object 
 1   condition          40216 non-null  object 
 2   intervention_type  40216 non-null  object 
 3   study_design       40216 non-null  object 
 4   sponsor_type       40216 non-null  object 
 5   enrollment         40216 non-null  float64
 6   enrollment_type    38792 non-null  object 
 7   gender             40216 non-null  object 
 8   location           40216 non-null  object 
 9   masking            40216 non-null  object 
 10  duration           40216 non-null  float64
 11  final_status       40216 non-null  int64  
dtypes: float64(2), int64(1), object(9)
memory usage: 3.7+ MB


In [50]:
categorical_cols = [
    'phase', 'condition', 'intervention_type', 'study_design',
    'sponsor_type', 'gender', 'location', 'masking'
]

# Dictionaries to store the mapping for each column
label_mappings = {}

for col in categorical_cols:
    # Get unique values and sort them for consistent mapping
    unique_vals = sorted(df_balanced[col].unique())
    mapping = {val: idx for idx, val in enumerate(unique_vals)}
    # Save mapping
    label_mappings[col] = mapping
    # Apply mapping to the column
    df_balanced[col + '_encoded'] = df_balanced[col].map(mapping)

# Display the mapping for each column
for col, mapping in label_mappings.items():
    print(f"Mapping for column '{col}':")
    for k, v in mapping.items():
        print(f"  {k}: {v}")
    print()

# Now df has new columns like 'phase_encoded', 'condition_encoded', etc.


Mapping for column 'phase':
  EARLY_PHASE1: 0
  PHASE1: 1
  PHASE1/PHASE2: 2
  PHASE2: 3
  PHASE2/PHASE3: 4
  PHASE3: 5
  PHASE4: 6

Mapping for column 'condition':
  "Wet" Age-Related Macular Degeneration: 0
  - HIV: 1
  - Medico-Economic Aspects (Evaluation of Medical Costs Related to the Three Strategies and Evaluation of Cost/Efficacy): 2
  11q-deleted Relapsed/Refractory Chronic Lymphocytic Leukaemia (CLL),: 3
  15q Duplication Syndrome: 4
  2009 H1N1 Influenza: 5
  22q11.2 Deletion Syndrome: 6
  2nd-line, 3rd-line and Greater Metastatic Pancreatic Cancer: 7
  3rd Line GIST: 8
  50 % Reduction of Delayed Gastric Emptying: 9
  714leukemia: 10
  A Combination of Adoptive T Cell Therapy and Ipilimumab Could Increase the Proportion of CR Patients, and Durability of Response: 11
  ABSORPTION: 12
  ACE Inhibitor Induced Angioedema: 13
  ACE Inhibitor-associated Angioedema: 14
  ACL - Anterior Cruciate Ligament Rupture: 15
  ACL Repair: 16
  ACL Surgery: 17
  ACOS (Fixed Airflow Obstruct


  Lymphoma, Diffuse Large-Cell B-cell: 5544
  Lymphoma, Extranodal NK-T-Cell: 5545
  Lymphoma, Follicular: 5546
  Lymphoma, Large B-Cell, Diffuse: 5547
  Lymphoma, Large Cell, Diffuse: 5548
  Lymphoma, Large-Cell: 5549
  Lymphoma, Large-Cell, Diffuse: 5550
  Lymphoma, Large-Cell, Ki-1: 5551
  Lymphoma, Low-Grade: 5552
  Lymphoma, Lymphoblastic: 5553
  Lymphoma, Malignant: 5554
  Lymphoma, Mantle Cell: 5555
  Lymphoma, Mantle-Cell: 5556
  Lymphoma, Mucosa-Associated Lymphoid Tissue: 5557
  Lymphoma, Non-Hodgkin: 5558
  Lymphoma, Non-Hodgkin's: 5559
  Lymphoma, Non-Hodgkin's, Adult: 5560
  Lymphoma, Non-Hodgkin; Leukemia, Chronic Lymphocytic: 5561
  Lymphoma, Non-Hodgkins: 5562
  Lymphoma, Small Cleaved-Cell, Follicular: 5563
  Lymphoma, Small Lymphocytic: 5564
  Lymphoma, T-Cell: 5565
  Lymphoma, T-Cell, Cutaneous: 5566
  Lymphoma, T-Cell, Peripheral: 5567
  Lymphomas: 5568
  Lymphomas Non-Hodgkin's B-Cell: 5569
  Lymphopenia: 5570
  Lymphoproliferative Disorder: 5571
  Lymphoprolifera

In [51]:
df_balanced.head()

,phase,condition,intervention_type,study_design,sponsor_type,enrollment,enrollment_type,gender,location,masking,duration,final_status,phase_encoded,condition_encoded,intervention_type_encoded,study_design_encoded,sponsor_type_encoded,gender_encoded,location_encoded,masking_encoded
0,PHASE3,Vulvovaginal Atrophy,DRUG,RANDOMIZED,INDUSTRY,550.0,ACTUAL,FEMALE,Alabama,TRIPLE,222.0,1,5,9940,6,1,3,1,37,4
1,PHASE2,Borderline Ovarian Serous Tumor,OTHER,UNKNOWN,OTHER,52.0,ACTUAL,FEMALE,California,NONE,4715.0,1,3,1409,8,2,6,1,208,1
2,PHASE4,Actinic Keratosis (AK),DRUG,RANDOMIZED,INDUSTRY,502.0,ACTUAL,ALL,other,NONE,427.0,1,6,157,6,1,3,0,1191,1
3,PHASE4,Health,DRUG,RANDOMIZED,OTHER,56.0,ACTUAL,MALE,D,TRIPLE,62.0,1,6,4047,6,1,6,2,307,4
4,PHASE4,Chronic Kidney Disease,DRUG,RANDOMIZED,OTHER_GOV,75.0,ACTUAL,ALL,Sao Paulo,DOUBLE,974.0,1,6,2079,6,1,7,0,971,0


In [52]:
from sklearn.preprocessing import StandardScaler

# Select the columns to standardize
scaler = StandardScaler()
df_balanced[['enrollment_standardized', 'duration_standardized']] = scaler.fit_transform(df_balanced[['enrollment', 'duration']])


In [53]:
df_balanced.head()

,phase,condition,intervention_type,study_design,sponsor_type,enrollment,enrollment_type,gender,location,masking,...,phase_encoded,condition_encoded,intervention_type_encoded,study_design_encoded,sponsor_type_encoded,gender_encoded,location_encoded,masking_encoded,enrollment_standardized,duration_standardized
0,PHASE3,Vulvovaginal Atrophy,DRUG,RANDOMIZED,INDUSTRY,550.0,ACTUAL,FEMALE,Alabama,TRIPLE,...,5,9940,6,1,3,1,37,4,0.209566,-0.920511
1,PHASE2,Borderline Ovarian Serous Tumor,OTHER,UNKNOWN,OTHER,52.0,ACTUAL,FEMALE,California,NONE,...,3,1409,8,2,6,1,208,1,-0.075062,4.004219
2,PHASE4,Actinic Keratosis (AK),DRUG,RANDOMIZED,INDUSTRY,502.0,ACTUAL,ALL,other,NONE,...,6,157,6,1,3,0,1191,1,0.182132,-0.695812
3,PHASE4,Health,DRUG,RANDOMIZED,OTHER,56.0,ACTUAL,MALE,D,TRIPLE,...,6,4047,6,1,6,2,307,4,-0.072776,-1.095885
4,PHASE4,Chronic Kidney Disease,DRUG,RANDOMIZED,OTHER_GOV,75.0,ACTUAL,ALL,Sao Paulo,DOUBLE,...,6,2079,6,1,7,0,971,0,-0.061917,-0.096251


In [54]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40216 entries, 0 to 40215
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   phase                      40216 non-null  object 
 1   condition                  40216 non-null  object 
 2   intervention_type          40216 non-null  object 
 3   study_design               40216 non-null  object 
 4   sponsor_type               40216 non-null  object 
 5   enrollment                 40216 non-null  float64
 6   enrollment_type            38792 non-null  object 
 7   gender                     40216 non-null  object 
 8   location                   40216 non-null  object 
 9   masking                    40216 non-null  object 
 10  duration                   40216 non-null  float64
 11  final_status               40216 non-null  int64  
 12  phase_encoded              40216 non-null  int64  
 13  condition_encoded          40216 non-null  int

In [55]:
columns_to_drop = [
    'phase', 'condition', 'intervention_type', 'study_design',
    'sponsor_type', 'enrollment_standardized', 'gender', 'location', 'masking', 'duration_standardized'
]

df_balanced = df_balanced.drop(columns=columns_to_drop)


In [56]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40216 entries, 0 to 40215
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   enrollment                 40216 non-null  float64
 1   enrollment_type            38792 non-null  object 
 2   duration                   40216 non-null  float64
 3   final_status               40216 non-null  int64  
 4   phase_encoded              40216 non-null  int64  
 5   condition_encoded          40216 non-null  int64  
 6   intervention_type_encoded  40216 non-null  int64  
 7   study_design_encoded       40216 non-null  int64  
 8   sponsor_type_encoded       40216 non-null  int64  
 9   gender_encoded             40216 non-null  int64  
 10  location_encoded           40216 non-null  int64  
 11  masking_encoded            40216 non-null  int64  
dtypes: float64(2), int64(9), object(1)
memory usage: 3.7+ MB


In [57]:
# Get a list of columns, excluding 'final_status'
cols = [col for col in df_balanced.columns if col != 'final_status']
# Add 'final_status' at the end
cols.append('final_status')
# Reorder the DataFrame
df_balanced = df_balanced[cols]

In [58]:
df_balanced.to_csv('trainable.csv', index=False)

In [59]:
df_balanced['final_status'].value_counts()

final_status
0    20216
1    20000
Name: count, dtype: int64